# Multi-Trait Education Experiment — Deep Analysis

Analyzing how 7 persona steering vectors affect:
- **Experiment A**: Student answer quality (steered students, judged by unsteered LLM + OpenAI)
- **Experiment B**: Judge grading bias (steered judges scoring real essays with ground truth)

Traits: evil, apathetic, hallucinating, humorous, impolite, optimistic, sycophantic

In [ ]:
import json
import numpy as np
import pandas as pd
from pathlib import Path
from collections import defaultdict

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', 120)

RESULTS_DIR = Path('results/multi_trait_20260206_104247')

TRAIT_OPPOSITES = {
    'evil': 'good', 'apathetic': 'empathetic', 'hallucinating': 'factual',
    'humorous': 'serious', 'impolite': 'polite', 'optimistic': 'pessimistic',
    'sycophantic': 'candid',
}
ALL_TRAITS = list(TRAIT_OPPOSITES.keys())

In [ ]:
# Load all data
def load_jsonl(path):
    records = []
    with open(path) as f:
        for line in f:
            if line.strip():
                records.append(json.loads(line))
    return pd.DataFrame(records)

# Exp A
answers_a = load_jsonl(RESULTS_DIR / 'experiment_a_student/generated_answers.jsonl')
scores_a = load_jsonl(RESULTS_DIR / 'experiment_a_student/scoring_results.jsonl')

# Exp B
scores_b = load_jsonl(RESULTS_DIR / 'experiment_b_judge/scoring_results.jsonl')
essays = load_jsonl(RESULTS_DIR / 'shared/sampled_essays.jsonl')

# Essay set info
with open(RESULTS_DIR / 'shared/essay_sets.json') as f:
    essay_sets_info = json.load(f)

# Parse trait/direction from student_type and judge_type
def parse_type(t):
    if t in ('unsteered', 'openai', 'real_student'):
        return t, 'none'
    parts = t.rsplit('_', 1)
    return parts[0], parts[1]

for df in [scores_a, answers_a]:
    df[['trait', 'direction']] = df['student_type'].apply(lambda x: pd.Series(parse_type(x)))

scores_b[['judge_trait', 'judge_direction']] = scores_b['judge_type'].apply(lambda x: pd.Series(parse_type(x)))

# Add answer length to answers
answers_a['answer_len'] = answers_a['answer'].str.len()

print(f'Exp A: {len(answers_a)} answers, {len(scores_a)} scores')
print(f'Exp B: {len(scores_b)} scores, {len(essays)} real essays')

---
# Part A: Student Trait Analysis
## A1. Overall Effect Sizes — Which traits matter most?

In [ ]:
# Compute effect sizes: mean score difference vs unsteered baseline
baseline = scores_a[scores_a['student_type'] == 'unsteered'].groupby('judge_type')['normalized_score'].mean()

effects = []
for st in scores_a['student_type'].unique():
    if st == 'unsteered':
        continue
    trait, direction = parse_type(st)
    for jt in ['unsteered', 'openai']:
        mean_score = scores_a[(scores_a['student_type'] == st) & (scores_a['judge_type'] == jt)]['normalized_score'].mean()
        effect = mean_score - baseline[jt]
        effects.append({
            'trait': trait, 'direction': direction, 'student_type': st,
            'judge_type': jt, 'mean_score': round(mean_score, 4),
            'baseline': round(baseline[jt], 4), 'effect': round(effect, 4),
        })

effects_df = pd.DataFrame(effects)

# Rank by absolute effect (averaged across both judges)
avg_effect = effects_df.groupby(['trait', 'direction'])['effect'].mean().reset_index()
avg_effect['abs_effect'] = avg_effect['effect'].abs()
avg_effect = avg_effect.sort_values('abs_effect', ascending=False)

print('=== Effect Sizes (avg across judges, sorted by magnitude) ===')
print(avg_effect[['trait', 'direction', 'effect', 'abs_effect']].to_string(index=False))
print(f'\nBaseline scores: unsteered judge={baseline["unsteered"]:.4f}, openai={baseline["openai"]:.4f}')

In [ ]:
# Visualization: effect size bar chart per trait (pos vs neg)
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

for idx, jt in enumerate(['unsteered', 'openai']):
    ax = axes[idx]
    subset = effects_df[effects_df['judge_type'] == jt]
    
    traits_order = []
    pos_effects = []
    neg_effects = []
    for trait in ALL_TRAITS:
        pos = subset[(subset['trait'] == trait) & (subset['direction'] == 'pos')]['effect'].values
        neg = subset[(subset['trait'] == trait) & (subset['direction'] == 'neg')]['effect'].values
        traits_order.append(trait)
        pos_effects.append(pos[0] if len(pos) > 0 else 0)
        neg_effects.append(neg[0] if len(neg) > 0 else 0)
    
    x = np.arange(len(traits_order))
    width = 0.35
    bars1 = ax.bar(x - width/2, pos_effects, width, label='pos (+trait)', color='#d62728', alpha=0.8)
    bars2 = ax.bar(x + width/2, neg_effects, width, label='neg (-trait/opposite)', color='#2ca02c', alpha=0.8)
    
    ax.set_xlabel('Trait')
    ax.set_ylabel('Effect on normalized score')
    ax.set_title(f'Student Steering Effect ({jt} judge)')
    ax.set_xticks(x)
    ax.set_xticklabels(traits_order, rotation=45, ha='right')
    ax.legend()
    ax.axhline(y=0, color='black', linewidth=0.5)
    ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'combined/student_effect_sizes.png', dpi=150, bbox_inches='tight')
plt.show()

## A2. Score Distributions — Consistency of steered students

In [ ]:
# Score distribution stats per student type
dist_stats = scores_a.groupby(['student_type', 'judge_type']).agg(
    mean=('normalized_score', 'mean'),
    std=('normalized_score', 'std'),
    median=('normalized_score', 'median'),
    q25=('normalized_score', lambda x: x.quantile(0.25)),
    q75=('normalized_score', lambda x: x.quantile(0.75)),
    min=('normalized_score', 'min'),
    max=('normalized_score', 'max'),
    count=('normalized_score', 'count'),
).round(4)

print('=== Score Distribution Stats (OpenAI judge) ===')
openai_dist = dist_stats.xs('openai', level='judge_type').sort_values('mean', ascending=False)
print(openai_dist.to_string())
print()
print('=== Score Distribution Stats (Unsteered judge) ===')
unst_dist = dist_stats.xs('unsteered', level='judge_type').sort_values('mean', ascending=False)
print(unst_dist.to_string())

In [ ]:
# Box plots: score distributions per student type
fig, axes = plt.subplots(1, 2, figsize=(18, 7), sharey=True)

for idx, jt in enumerate(['openai', 'unsteered']):
    ax = axes[idx]
    subset = scores_a[scores_a['judge_type'] == jt]
    
    # Order by mean score
    order = subset.groupby('student_type')['normalized_score'].mean().sort_values(ascending=False).index
    
    data = [subset[subset['student_type'] == st]['normalized_score'].values for st in order]
    bp = ax.boxplot(data, labels=order, vert=True, patch_artist=True)
    
    # Color by direction
    for i, st in enumerate(order):
        _, d = parse_type(st)
        color = '#d62728' if d == 'pos' else '#2ca02c' if d == 'neg' else '#1f77b4'
        bp['boxes'][i].set_facecolor(color)
        bp['boxes'][i].set_alpha(0.6)
    
    ax.set_title(f'Score Distributions ({jt} judge)')
    ax.set_ylabel('Normalized Score')
    ax.tick_params(axis='x', rotation=90)
    ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'combined/student_score_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

## A3. Per-Set Analysis — Which question types are most affected?

In [ ]:
# Effect per set: how does each trait affect scores on different question types?
set_topics = {int(k): v['topic'] for k, v in essay_sets_info.items()}

baseline_per_set = scores_a[scores_a['student_type'] == 'unsteered'].groupby(
    ['set_id', 'judge_type']
)['normalized_score'].mean()

set_effects = []
for st in scores_a['student_type'].unique():
    if st == 'unsteered':
        continue
    trait, direction = parse_type(st)
    for sid in scores_a['set_id'].unique():
        for jt in ['openai']:
            mean = scores_a[(scores_a['student_type'] == st) & 
                           (scores_a['set_id'] == sid) &
                           (scores_a['judge_type'] == jt)]['normalized_score'].mean()
            bl = baseline_per_set.get((sid, jt), 0)
            set_effects.append({
                'trait': trait, 'direction': direction, 'student_type': st,
                'set_id': sid, 'topic': set_topics.get(sid, '?'),
                'mean_score': round(mean, 4), 'baseline': round(bl, 4),
                'effect': round(mean - bl, 4),
            })

set_effects_df = pd.DataFrame(set_effects)

# Heatmap: trait_pos effect per set (OpenAI judge)
pos_effects = set_effects_df[set_effects_df['direction'] == 'pos'].pivot_table(
    values='effect', index='trait', columns='set_id', aggfunc='mean'
)

fig, axes = plt.subplots(1, 2, figsize=(18, 6))

for idx, (direction, title) in enumerate([('pos', 'Positive Steering (+trait)'), ('neg', 'Negative Steering (-trait/opposite)')]):
    ax = axes[idx]
    pivot = set_effects_df[set_effects_df['direction'] == direction].pivot_table(
        values='effect', index='trait', columns='set_id', aggfunc='mean'
    )
    im = ax.imshow(pivot.values, cmap='RdYlGn', aspect='auto', vmin=-0.6, vmax=0.3)
    ax.set_xticks(range(len(pivot.columns)))
    ax.set_xticklabels(pivot.columns)
    ax.set_yticks(range(len(pivot.index)))
    ax.set_yticklabels(pivot.index)
    ax.set_xlabel('Essay Set')
    ax.set_title(title)
    
    # Annotate cells
    for i in range(len(pivot.index)):
        for j in range(len(pivot.columns)):
            val = pivot.values[i, j]
            ax.text(j, i, f'{val:.2f}', ha='center', va='center', fontsize=8,
                   color='white' if abs(val) > 0.3 else 'black')

plt.colorbar(im, ax=axes, shrink=0.8, label='Effect on normalized score')
plt.suptitle('Student Steering Effect by Essay Set (OpenAI judge)', fontsize=14)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'combined/student_effect_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Which sets are most/least affected by each trait?
print('=== Most Affected Sets Per Trait (pos direction, OpenAI judge) ===')
for trait in ALL_TRAITS:
    sub = set_effects_df[(set_effects_df['trait'] == trait) & (set_effects_df['direction'] == 'pos')]
    worst = sub.loc[sub['effect'].idxmin()]
    best = sub.loc[sub['effect'].idxmax()]
    print(f'\n{trait} (+{trait}):')
    print(f'  Worst: Set {worst["set_id"]} ({worst["topic"]}) effect={worst["effect"]:+.3f}')
    print(f'  Best:  Set {best["set_id"]} ({best["topic"]}) effect={best["effect"]:+.3f}')

## A4. Answer Length Analysis

In [ ]:
# Answer length by student type
len_stats = answers_a.groupby('student_type')['answer_len'].agg(['mean', 'std', 'median']).round(1)
len_stats = len_stats.sort_values('mean', ascending=False)
print('=== Answer Length by Student Type ===')
print(len_stats.to_string())

# Merge answer length with scores
scores_with_len = scores_a.merge(
    answers_a[['set_id', 'sample_id', 'student_type', 'answer_len']],
    on=['set_id', 'sample_id', 'student_type'],
    how='left'
)

# Correlation between length and score
print('\n=== Length-Score Correlation (by student type, OpenAI judge) ===')
openai_scores = scores_with_len[scores_with_len['judge_type'] == 'openai']
for st in sorted(openai_scores['student_type'].unique()):
    sub = openai_scores[openai_scores['student_type'] == st]
    if len(sub) > 2 and sub['answer_len'].std() > 0:
        corr = sub['answer_len'].corr(sub['normalized_score'])
        print(f'  {st:25s} r={corr:+.3f}  (mean_len={sub["answer_len"].mean():.0f})')

In [ ]:
# Scatter: answer length vs score for selected traits
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

student_types_to_plot = ['unsteered'] + [f'{t}_pos' for t in ALL_TRAITS]
openai_with_len = scores_with_len[scores_with_len['judge_type'] == 'openai']

for idx, st in enumerate(student_types_to_plot[:8]):
    ax = axes[idx]
    sub = openai_with_len[openai_with_len['student_type'] == st]
    ax.scatter(sub['answer_len'], sub['normalized_score'], alpha=0.4, s=20)
    ax.set_title(st, fontsize=10)
    ax.set_xlabel('Answer length (chars)')
    ax.set_ylabel('Normalized score')
    if len(sub) > 2 and sub['answer_len'].std() > 0:
        corr = sub['answer_len'].corr(sub['normalized_score'])
        ax.text(0.05, 0.95, f'r={corr:.2f}', transform=ax.transAxes, va='top', fontsize=9)

plt.suptitle('Answer Length vs Score (OpenAI judge)', fontsize=14)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'combined/length_vs_score.png', dpi=150, bbox_inches='tight')
plt.show()

---
# Part B: Judge Trait Analysis
## B1. Judge Performance vs Ground Truth

In [ ]:
# Overall judge performance metrics
judge_perf = []
for jt in scores_b['judge_type'].unique():
    sub = scores_b[scores_b['judge_type'] == jt]
    pred = sub['raw_score'].values
    gt = sub['ground_truth_score'].values
    
    bias = pred - gt
    judge_perf.append({
        'judge_type': jt,
        'mean_bias': round(np.mean(bias), 3),
        'std_bias': round(np.std(bias), 3),
        'mae': round(np.mean(np.abs(bias)), 3),
        'rmse': round(np.sqrt(np.mean(bias**2)), 3),
        'mean_pred': round(np.mean(pred), 3),
        'mean_gt': round(np.mean(gt), 3),
        'pred_std': round(np.std(pred), 3),
        'gt_std': round(np.std(gt), 3),
        'corr': round(np.corrcoef(pred, gt)[0,1], 3) if np.std(pred) > 0 else 0,
        'count': len(sub),
    })

judge_perf_df = pd.DataFrame(judge_perf).sort_values('mae')
print('=== Judge Performance vs Ground Truth (sorted by MAE) ===')
print(judge_perf_df.to_string(index=False))

In [ ]:
# QWK from the experiment results
qwk_df = pd.read_csv(RESULTS_DIR / 'experiment_b_judge/qwk_scores.csv')

# Merge with parsed trait info
qwk_df[['trait', 'direction']] = qwk_df['judge_type'].apply(lambda x: pd.Series(parse_type(x)))

print('=== QWK Scores (sorted by mean_qwk) ===')
qwk_display = qwk_df[['judge_type', 'trait', 'direction', 'mean_qwk', 'std_qwk', 'mean_bias', 'mae']].sort_values('mean_qwk', ascending=False)
print(qwk_display.to_string(index=False))

In [ ]:
# Visualization: judge bias and QWK
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart: mean bias per judge
ax = axes[0]
perf_sorted = judge_perf_df.sort_values('mean_bias')
colors = []
for jt in perf_sorted['judge_type']:
    _, d = parse_type(jt)
    colors.append('#d62728' if d == 'pos' else '#2ca02c' if d == 'neg' else '#1f77b4' if jt == 'unsteered' else '#ff7f0e')
ax.barh(range(len(perf_sorted)), perf_sorted['mean_bias'], color=colors, alpha=0.8)
ax.set_yticks(range(len(perf_sorted)))
ax.set_yticklabels(perf_sorted['judge_type'], fontsize=9)
ax.set_xlabel('Mean Bias (predicted - ground truth)')
ax.set_title('Judge Bias')
ax.axvline(x=0, color='black', linewidth=0.5)
ax.grid(axis='x', alpha=0.3)

# Bar chart: mean QWK per judge
ax = axes[1]
qwk_sorted = qwk_df.sort_values('mean_qwk', ascending=True)
colors = []
for jt in qwk_sorted['judge_type']:
    _, d = parse_type(jt)
    colors.append('#d62728' if d == 'pos' else '#2ca02c' if d == 'neg' else '#1f77b4' if jt == 'unsteered' else '#ff7f0e')
ax.barh(range(len(qwk_sorted)), qwk_sorted['mean_qwk'], color=colors, alpha=0.8)
ax.set_yticks(range(len(qwk_sorted)))
ax.set_yticklabels(qwk_sorted['judge_type'], fontsize=9)
ax.set_xlabel('Mean QWK')
ax.set_title('Judge Agreement with Ground Truth (QWK)')
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'combined/judge_performance.png', dpi=150, bbox_inches='tight')
plt.show()

## B2. Per-Set Judge Quality — Which question types are hardest to judge?

In [ ]:
# Per-set QWK heatmap
qwk_set_cols = [c for c in qwk_df.columns if c.startswith('qwk_set_')]
qwk_set_data = qwk_df.set_index('judge_type')[qwk_set_cols]
qwk_set_data.columns = [int(c.replace('qwk_set_', '')) for c in qwk_set_data.columns]

fig, ax = plt.subplots(figsize=(14, 8))
im = ax.imshow(qwk_set_data.values, cmap='RdYlGn', aspect='auto', vmin=-0.5, vmax=1.0)
ax.set_xticks(range(len(qwk_set_data.columns)))
ax.set_xticklabels(qwk_set_data.columns)
ax.set_yticks(range(len(qwk_set_data.index)))
ax.set_yticklabels(qwk_set_data.index, fontsize=9)
ax.set_xlabel('Essay Set')
ax.set_title('QWK by Judge Type and Essay Set')

for i in range(len(qwk_set_data.index)):
    for j in range(len(qwk_set_data.columns)):
        val = qwk_set_data.values[i, j]
        ax.text(j, i, f'{val:.2f}', ha='center', va='center', fontsize=7,
               color='white' if val < 0 or val > 0.7 else 'black')

plt.colorbar(im, shrink=0.8, label='QWK')
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'combined/judge_qwk_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Per-set bias heatmap
bias_set_cols = [c for c in qwk_df.columns if c.startswith('bias_set_')]
bias_set_data = qwk_df.set_index('judge_type')[bias_set_cols]
bias_set_data.columns = [int(c.replace('bias_set_', '')) for c in bias_set_data.columns]

fig, ax = plt.subplots(figsize=(14, 8))
im = ax.imshow(bias_set_data.values, cmap='RdBu_r', aspect='auto', vmin=-1.5, vmax=1.5)
ax.set_xticks(range(len(bias_set_data.columns)))
ax.set_xticklabels(bias_set_data.columns)
ax.set_yticks(range(len(bias_set_data.index)))
ax.set_yticklabels(bias_set_data.index, fontsize=9)
ax.set_xlabel('Essay Set')
ax.set_title('Bias by Judge Type and Essay Set (predicted - ground truth)')

for i in range(len(bias_set_data.index)):
    for j in range(len(bias_set_data.columns)):
        val = bias_set_data.values[i, j]
        ax.text(j, i, f'{val:.1f}', ha='center', va='center', fontsize=7,
               color='white' if abs(val) > 1.0 else 'black')

plt.colorbar(im, shrink=0.8, label='Bias')
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'combined/judge_bias_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

## B3. Score Compression — Do steered judges lose score range?

In [ ]:
# Score range usage per judge
score_range_stats = scores_b.groupby('judge_type').agg(
    pred_std=('raw_score', 'std'),
    pred_range=('raw_score', lambda x: x.max() - x.min()),
    unique_scores=('raw_score', 'nunique'),
    mode_score=('raw_score', lambda x: x.mode()[0] if len(x.mode()) > 0 else None),
    mode_pct=('raw_score', lambda x: (x == x.mode()[0]).mean() if len(x.mode()) > 0 else 0),
).round(3)

print('=== Score Range Usage by Judge (compression analysis) ===')
print(score_range_stats.sort_values('pred_std', ascending=False).to_string())
print('\nLower std = more compressed scoring. Higher mode_pct = more repetitive.')

---
# Part C: Cross-Experiment Analysis
## C1. Student Effect vs Judge Bias Correlation

In [ ]:
# For each trait: student effect (Exp A) vs judge bias (Exp B)
comparison = pd.read_csv(RESULTS_DIR / 'combined/trait_comparison.csv')

# Get student pos effect (avg of both judges) and judge pos bias
cross_data = []
for trait in ALL_TRAITS:
    # Student effect from Exp A (average of both judges)
    student_rows = comparison[(comparison['trait'] == trait) & (comparison['experiment'] == 'A_student')]
    student_pos_effect = student_rows['pos_effect'].mean()
    student_neg_effect = student_rows['neg_effect'].mean()
    
    # Judge bias from Exp B
    judge_row = comparison[(comparison['trait'] == trait) & (comparison['experiment'] == 'B_judge')]
    if len(judge_row) > 0:
        judge_pos_effect = judge_row['pos_effect'].values[0]
        judge_neg_effect = judge_row['neg_effect'].values[0]
    else:
        judge_pos_effect = judge_neg_effect = 0
    
    cross_data.append({
        'trait': trait, 'opposite': TRAIT_OPPOSITES[trait],
        'student_pos_effect': round(student_pos_effect, 4),
        'student_neg_effect': round(student_neg_effect, 4),
        'judge_pos_bias': round(judge_pos_effect, 4),
        'judge_neg_bias': round(judge_neg_effect, 4),
    })

cross_df = pd.DataFrame(cross_data)
print('=== Cross-Experiment: Student Effect vs Judge Bias ===')
print(cross_df.to_string(index=False))

# Scatter plot
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for idx, direction in enumerate(['pos', 'neg']):
    ax = axes[idx]
    x = cross_df[f'student_{direction}_effect']
    y = cross_df[f'judge_{direction}_bias']
    
    ax.scatter(x, y, s=100, zorder=5)
    for i, trait in enumerate(cross_df['trait']):
        ax.annotate(trait, (x.iloc[i], y.iloc[i]), textcoords='offset points',
                   xytext=(8, 5), fontsize=10)
    
    ax.set_xlabel(f'Student Effect ({direction})')
    ax.set_ylabel(f'Judge Bias ({direction})')
    ax.set_title(f'{direction.upper()} direction')
    ax.axhline(y=0, color='gray', linewidth=0.5)
    ax.axvline(x=0, color='gray', linewidth=0.5)
    ax.grid(alpha=0.3)
    
    if x.std() > 0 and y.std() > 0:
        corr = np.corrcoef(x, y)[0, 1]
        ax.text(0.05, 0.95, f'r={corr:.2f}', transform=ax.transAxes, va='top', fontsize=11)

plt.suptitle('Does a trait that hurts students also bias judges?', fontsize=14)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'combined/cross_experiment_correlation.png', dpi=150, bbox_inches='tight')
plt.show()

## C2. Trait Clustering — Which traits have similar effects?

In [ ]:
# Build feature vectors per trait: [student_pos_effect_per_set..., student_neg_effect_per_set..., judge_bias, qwk]
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist

trait_features = []
set_ids = sorted(scores_a['set_id'].unique())

for trait in ALL_TRAITS:
    feats = {'trait': trait}
    
    # Student effects per set (pos and neg)
    for direction in ['pos', 'neg']:
        st = f'{trait}_{direction}'
        for sid in set_ids:
            mean = scores_a[
                (scores_a['student_type'] == st) & 
                (scores_a['set_id'] == sid) & 
                (scores_a['judge_type'] == 'openai')
            ]['normalized_score'].mean()
            feats[f's_{direction}_set{sid}'] = mean
    
    # Judge bias (pos and neg)
    for direction in ['pos', 'neg']:
        jt = f'{trait}_{direction}'
        sub = scores_b[scores_b['judge_type'] == jt]
        feats[f'j_{direction}_bias'] = (sub['raw_score'] - sub['ground_truth_score']).mean()
        feats[f'j_{direction}_mae'] = (sub['raw_score'] - sub['ground_truth_score']).abs().mean()
    
    trait_features.append(feats)

trait_feat_df = pd.DataFrame(trait_features).set_index('trait')

# Dendrogram
fig, ax = plt.subplots(figsize=(10, 6))
linkage_matrix = linkage(pdist(trait_feat_df.values, 'euclidean'), method='ward')
dendrogram(linkage_matrix, labels=trait_feat_df.index.tolist(), ax=ax, leaf_font_size=12)
ax.set_title('Trait Similarity (hierarchical clustering on student + judge effects)')
ax.set_ylabel('Distance')
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'combined/trait_clustering.png', dpi=150, bbox_inches='tight')
plt.show()

## C3. Asymmetry Analysis — Is positive always worse?

In [ ]:
# For each trait × set: compare pos vs neg effect
asymmetry_data = []
for trait in ALL_TRAITS:
    for sid in set_ids:
        for jt in ['openai']:
            pos_mean = scores_a[
                (scores_a['student_type'] == f'{trait}_pos') &
                (scores_a['set_id'] == sid) & (scores_a['judge_type'] == jt)
            ]['normalized_score'].mean()
            neg_mean = scores_a[
                (scores_a['student_type'] == f'{trait}_neg') &
                (scores_a['set_id'] == sid) & (scores_a['judge_type'] == jt)
            ]['normalized_score'].mean()
            bl = baseline_per_set.get((sid, jt), 0)
            
            asymmetry_data.append({
                'trait': trait, 'set_id': sid,
                'pos_effect': round(pos_mean - bl, 4),
                'neg_effect': round(neg_mean - bl, 4),
                'asymmetry': round((pos_mean - bl) - (neg_mean - bl), 4),  # pos - neg
                'neg_better_than_pos': neg_mean > pos_mean,
            })

asym_df = pd.DataFrame(asymmetry_data)

# Count reversals: cases where neg direction scores LOWER than pos
print('=== Asymmetry: How often is neg (opposite) better than pos? ===')
for trait in ALL_TRAITS:
    sub = asym_df[asym_df['trait'] == trait]
    pct_neg_better = sub['neg_better_than_pos'].mean() * 100
    mean_asym = sub['asymmetry'].mean()
    print(f'  {trait:20s} neg>pos in {pct_neg_better:.0f}% of sets, mean asymmetry={mean_asym:+.3f}')

print('\n(Positive asymmetry = pos is worse than neg, as expected)')
print('(Reversals = cases where opposite-steered student scores LOWER)')

---
# Part D: Cherry-Picked Cases
## D1. Student Cases — Answers that reveal trait patterns

In [ ]:
# Merge scores with answers for cherry picking
merged_a = scores_a[scores_a['judge_type'] == 'openai'].merge(
    answers_a[['set_id', 'sample_id', 'student_type', 'answer', 'answer_len']],
    on=['set_id', 'sample_id', 'student_type'],
    how='left'
)

def show_case(row, label=''):
    print(f'\n{"=" * 80}')
    print(f'{label}')
    print(f'Set {row["set_id"]} | {row["student_type"]} | Score: {row["raw_score"]}/{row["score_range"][1] if isinstance(row["score_range"], list) else "?"} (norm={row["normalized_score"]:.2f})')
    print(f'Answer ({row["answer_len"]:.0f} chars):')
    print(f'{row["answer"][:500]}')
    if row['answer_len'] > 500:
        print(f'... [{row["answer_len"]:.0f} total chars]')
    print(f'{"=" * 80}')

In [ ]:
# Find most extreme cases: steered student scores much worse than unsteered
# Compare each steered answer to the unsteered answer for the same (set_id, sample_id)
unsteered_scores = merged_a[merged_a['student_type'] == 'unsteered'][['set_id', 'sample_id', 'normalized_score']].rename(
    columns={'normalized_score': 'unsteered_score'}
)

steered_vs_unsteered = merged_a[merged_a['student_type'] != 'unsteered'].merge(
    unsteered_scores, on=['set_id', 'sample_id'], how='left'
)
steered_vs_unsteered['score_drop'] = steered_vs_unsteered['normalized_score'] - steered_vs_unsteered['unsteered_score']

print('=== WORST CASES: Biggest score drops from steering ===')
worst = steered_vs_unsteered.nsmallest(5, 'score_drop')
for _, row in worst.iterrows():
    show_case(row, f'WORST DROP: {row["student_type"]} scored {row["score_drop"]:+.2f} vs unsteered')

In [ ]:
print('=== BEST CASES: Steering unexpectedly improves score ===')
best = steered_vs_unsteered.nlargest(5, 'score_drop')
for _, row in best.iterrows():
    show_case(row, f'BEST IMPROVEMENT: {row["student_type"]} scored {row["score_drop"]:+.2f} vs unsteered')

In [ ]:
# Trait-specific cherry picks: show one example per trait where the trait is clearly visible
print('=== TRAIT-REVEALING EXAMPLES (pos direction, lowest scoring) ===')
for trait in ALL_TRAITS:
    sub = merged_a[(merged_a['student_type'] == f'{trait}_pos')].nsmallest(1, 'normalized_score')
    if len(sub) > 0:
        row = sub.iloc[0]
        show_case(row, f'TRAIT: {trait} (+pos) — lowest scoring example')

In [ ]:
# Side-by-side: same question, unsteered vs humorous_pos (biggest effect trait)
print('=== SIDE-BY-SIDE: Unsteered vs humorous_pos (same prompt) ===')
# Pick a case where humorous_pos scored much lower
humor_drops = steered_vs_unsteered[steered_vs_unsteered['student_type'] == 'humorous_pos'].nsmallest(3, 'score_drop')

for _, humor_row in humor_drops.iterrows():
    sid, samp = humor_row['set_id'], humor_row['sample_id']
    unst_row = merged_a[(merged_a['set_id'] == sid) & (merged_a['sample_id'] == samp) & (merged_a['student_type'] == 'unsteered')]
    if len(unst_row) > 0:
        print(f'\n{"#" * 80}')
        print(f'Set {sid}, Sample {samp}')
        print(f'\n--- UNSTEERED (score={unst_row.iloc[0]["normalized_score"]:.2f}) ---')
        print(unst_row.iloc[0]['answer'][:400])
        print(f'\n--- HUMOROUS_POS (score={humor_row["normalized_score"]:.2f}) ---')
        print(humor_row['answer'][:400])
        print()

## D2. Judge Cases — How steered judges misjudge

In [ ]:
# Find cases where steered judges deviate most from ground truth
scores_b['error'] = scores_b['raw_score'] - scores_b['ground_truth_score']
scores_b['abs_error'] = scores_b['error'].abs()

# Merge with essay text
merged_b = scores_b.merge(
    essays[['essay_id', 'essay_text', 'avg_score']],
    left_on='essay_id', right_on='essay_id',
    how='left'
)

def show_judge_case(row, label=''):
    print(f'\n{"=" * 80}')
    print(f'{label}')
    print(f'Essay {row["essay_id"]} (Set {row["set_id"]}) | Judge: {row["judge_type"]}')
    print(f'Predicted: {row["raw_score"]} | Ground Truth: {row["ground_truth_score"]} | Error: {row["error"]:+.1f}')
    print(f'Essay text ({len(str(row["essay_text"]))} chars):')
    print(str(row['essay_text'])[:400])
    if len(str(row['essay_text'])) > 400:
        print(f'... [{len(str(row["essay_text"]))} total chars]')
    print(f'{"=" * 80}')

In [ ]:
# Biggest overscoring by steered judges (excluding openai)
steered_judges = merged_b[~merged_b['judge_type'].isin(['openai', 'unsteered'])]

print('=== BIGGEST OVERSCORING by steered judges ===')
overscored = steered_judges.nlargest(5, 'error')
for _, row in overscored.iterrows():
    show_judge_case(row, f'OVERSCORED by {row["error"]:+.1f}')

print('\n\n=== BIGGEST UNDERSCORING by steered judges ===')
underscored = steered_judges.nsmallest(5, 'error')
for _, row in underscored.iterrows():
    show_judge_case(row, f'UNDERSCORED by {row["error"]:+.1f}')

In [ ]:
# Compare: same essay, different judges — find max disagreement
score_matrix_b = pd.read_csv(RESULTS_DIR / 'experiment_b_judge/score_matrix.csv')

# Find essays with biggest spread across judges
raw_cols = [c for c in score_matrix_b.columns if c.endswith('_raw')]
score_matrix_b['judge_spread'] = score_matrix_b[raw_cols].max(axis=1) - score_matrix_b[raw_cols].min(axis=1)
score_matrix_b['judge_std'] = score_matrix_b[raw_cols].std(axis=1)

print('=== MOST CONTROVERSIAL ESSAYS (biggest spread across judges) ===')
controversial = score_matrix_b.nlargest(5, 'judge_spread')
for _, row in controversial.iterrows():
    eid = row.get('essay_id', '?')
    gt = row.get('ground_truth', '?')
    print(f'\nEssay {eid} (Set {row["set_id"]}) | GT={gt} | Spread={row["judge_spread"]:.0f} | Std={row["judge_std"]:.2f}')
    scores = {c.replace('_raw', ''): row[c] for c in raw_cols if pd.notna(row[c])}
    scores_sorted = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for judge, score in scores_sorted:
        marker = ' <<<' if abs(score - gt) > 1.5 else ''
        print(f'  {judge:25s}: {score:.0f}{marker}')

In [ ]:
# Per-trait: sycophantic judges — do they always score higher? (lenient)
# vs evil judges — do they score lower? (harsh)
print('=== JUDGE LENIENCY PATTERN ===')
print('\nMean raw score by judge type (vs ground truth mean):')
gt_mean = scores_b['ground_truth_score'].mean()
print(f'  Ground truth mean: {gt_mean:.2f}\n')

leniency = scores_b.groupby('judge_type').agg(
    mean_pred=('raw_score', 'mean'),
    mean_error=('error', 'mean'),
).round(3).sort_values('mean_pred', ascending=False)

for jt, row in leniency.iterrows():
    direction = 'LENIENT' if row['mean_error'] > 0.1 else 'HARSH' if row['mean_error'] < -0.1 else 'NEUTRAL'
    print(f'  {jt:25s} pred={row["mean_pred"]:.2f}  bias={row["mean_error"]:+.2f}  [{direction}]')

---
# Summary Table

In [ ]:
# Final summary: one row per trait with key metrics from both experiments
summary = []
for trait in ALL_TRAITS:
    row = {'trait': trait, 'opposite': TRAIT_OPPOSITES[trait]}
    
    # Student effects (avg of both judges)
    for d in ['pos', 'neg']:
        st = f'{trait}_{d}'
        mean = scores_a[scores_a['student_type'] == st]['normalized_score'].mean()
        bl = scores_a[scores_a['student_type'] == 'unsteered']['normalized_score'].mean()
        row[f'student_{d}_effect'] = round(mean - bl, 3)
    
    # Judge metrics
    for d in ['pos', 'neg']:
        jt = f'{trait}_{d}'
        qwk_row = qwk_df[qwk_df['judge_type'] == jt]
        if len(qwk_row) > 0:
            row[f'judge_{d}_qwk'] = qwk_row['mean_qwk'].values[0]
            row[f'judge_{d}_bias'] = qwk_row['mean_bias'].values[0]
    
    # Answer length effect
    pos_len = answers_a[answers_a['student_type'] == f'{trait}_pos']['answer_len'].mean()
    unst_len = answers_a[answers_a['student_type'] == 'unsteered']['answer_len'].mean()
    row['len_change_pct'] = round((pos_len - unst_len) / unst_len * 100, 1)
    
    summary.append(row)

summary_df = pd.DataFrame(summary)
print('=== COMPREHENSIVE TRAIT SUMMARY ===')
print(summary_df.to_string(index=False))